In [1]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator

print(tf.__version__)

root_path = './'
dataset_path = os.path.join(root_path, 'faces-data-new')

print(dataset_path)

models_path = os.path.join(root_path, 'saved_models_new')
if not os.path.exists(models_path):
    os.mkdir(models_path)


tgt_pth = os.path.join(root_path, 'visualize_dcgan-v20')

if not os.path.exists(tgt_pth):
    os.mkdir(tgt_pth)



BATCH_SIZE = 32
img_rows, img_cols, img_depth = (196, 196, 3)


def rescaleFn(img):
    return img / 127.5 - 1


datagen=ImageDataGenerator(preprocessing_function=rescaleFn)


train_generator = datagen.flow_from_directory(
    directory=dataset_path,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=True,
    target_size=(img_rows, img_cols)
)

Using TensorFlow backend.


1.13.1
./faces-data-new
Found 3005 images belonging to 1 classes.


In [2]:
img_shape = (196, 196, 3)
img_len = np.prod(img_shape)
latent_dim = img_len
noise_len = 100
input_dim = noise_len
ngf = 64  # generator
ndf = 16  # discriminator


def build_discriminator():
    return keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=ndf,
            kernel_size=(4, 4),
            strides=2,
            input_shape=img_shape,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 2
        keras.layers.Conv2D(
            filters=ndf * 2,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=ndf * 4,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 4
        keras.layers.Conv2D(
            filters=ndf * 8,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.leaky_relu),
        
        # conv block 5
        keras.layers.Conv2D(
            filters=1,
            kernel_size=(9, 9),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        
        #output
        keras.layers.Activation(tf.nn.sigmoid),
        
        #flatten
        keras.layers.Flatten()
    ])


def build_generator():
    
    return keras.Sequential([
        # reshape 1d to 3d
        keras.layers.Reshape((10, 10, 1), input_shape=(input_dim,)),
        
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=ngf * 8,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=ngf * 4,
            kernel_size=(3, 3),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=ngf * 2,
            kernel_size=(3, 3),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=ngf,
            kernel_size=(3, 3),
            strides=1,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 5
        keras.layers.Conv2DTranspose(
            filters=ngf,
            kernel_size=(3, 3),
            strides=1,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 6
        keras.layers.Conv2DTranspose(
            filters=ngf,
            kernel_size=(3, 3),
            strides=1,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Activation(tf.nn.relu),
        
        # transpose conv block 7
        keras.layers.Conv2DTranspose(
            filters=3,
            kernel_size=(4, 4),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.he_normal()
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])


discriminator = build_discriminator()
generator = build_generator()

generator.summary()
discriminator.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 10, 10, 1)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 22, 22, 512)       8192      
_________________________________________________________________
batch_normalization_4 (Batch (None, 22, 22, 512)       2048      
_________________________________________________________________
activation_6 (Activation)    (None, 22, 22, 512)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 45, 45, 256)       1179648   
_________________________________________________________________
batch_normalization_5 (Batch (None, 45, 45, 256)       1024      
_________________________________________________________________
acti

In [3]:
GLR = 0.0002  # generator
DLR = 0.0002  # discriminator
LR = 0.0002


optim = keras.optimizers.Adam(LR, beta_1=0.5)


discriminator.compile(
    optimizer=optim,
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

# noise
z = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
x = generator(z)
out = discriminator(x)

# GAN
gan = keras.models.Model(inputs=z, outputs=out)

gan.compile(
    optimizer=optim,
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy'])

In [4]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(4, 4, figsize=(20, 18))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(
                    0.5 * z_real[r_real * 4 + col] + 0.5
                )
            else:
                cell.imshow(
                    0.5 * z_fake[r_fake * 4 + col] + 0.5
                )

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

In [ ]:
EPOCHS = 10251
STEPS = 1


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


# to be visualized
vis_noise = np.random.normal(size=(8, noise_len,))

# fake and real label ( Label Smoothing & Label Flipping )
y_fake = np.ones(shape=(BATCH_SIZE,))
y_real = np.zeros(shape=(BATCH_SIZE,))


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS * 4):
        
        z = np.random.normal(loc=0.0, scale=0.2, size=(BATCH_SIZE, noise_len))
        x_fake = generator.predict(z)
        x_real = next(train_generator)
        
        if x_real.shape[0] < BATCH_SIZE:
            x_real = next(train_generator)

        # add noise
        x_real = x_real + np.random.normal(loc=0, scale=0.01, size=x_real.shape)
        x_fake = x_fake + np.random.normal(loc=0, scale=0.01, size=x_fake.shape)

        # train
        loss_2, acc_2 = discriminator.train_on_batch(x_real, y_real)
        loss_1, acc_1 = discriminator.train_on_batch(x_fake, y_fake)

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS * 1):

        z = np.random.normal(loc=0.0, scale=0.2, size=(BATCH_SIZE, noise_len))

        # train
        loss_1, acc_1 = gan.train_on_batch(z, y_real)
        
        loss.append(loss_1)
        acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("E: {}, [D ACC: %{:.2f}], [D LOSS: {:.2f}], [G ACC: %{:.2f}], [G LOSS: {:.2f}]".format(
          e,
          train_acc_d[-1] * 100,
          train_loss_d[-1] * 100,
          train_acc_g[-1] * 100,
          train_loss_g[-1] * 100
      ))

    if e % 250 == 0:
        ## visualize results
        viz_fake = generator.predict(vis_noise)
        x_real = next(train_generator)
        visualizeGAN(e, x_real, viz_fake)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


E: 0, [D ACC: %65.23], [D LOSS: 61.11], [G ACC: %12.50], [G LOSS: 127.80]
E: 1, [D ACC: %87.11], [D LOSS: 36.16], [G ACC: %9.38], [G LOSS: 186.79]
E: 2, [D ACC: %95.70], [D LOSS: 21.02], [G ACC: %0.00], [G LOSS: 241.23]
E: 3, [D ACC: %94.53], [D LOSS: 18.04], [G ACC: %3.12], [G LOSS: 234.04]
E: 4, [D ACC: %96.48], [D LOSS: 15.55], [G ACC: %0.00], [G LOSS: 213.39]
E: 5, [D ACC: %97.27], [D LOSS: 15.75], [G ACC: %3.12], [G LOSS: 198.65]
E: 6, [D ACC: %98.83], [D LOSS: 10.72], [G ACC: %3.12], [G LOSS: 227.28]
E: 7, [D ACC: %96.88], [D LOSS: 11.57], [G ACC: %3.12], [G LOSS: 171.81]
E: 8, [D ACC: %100.00], [D LOSS: 5.73], [G ACC: %12.50], [G LOSS: 166.94]
E: 9, [D ACC: %99.61], [D LOSS: 6.29], [G ACC: %6.25], [G LOSS: 159.00]
E: 10, [D ACC: %98.44], [D LOSS: 8.16], [G ACC: %9.38], [G LOSS: 176.98]
E: 11, [D ACC: %98.05], [D LOSS: 7.62], [G ACC: %6.25], [G LOSS: 177.15]
E: 12, [D ACC: %99.22], [D LOSS: 5.81], [G ACC: %9.38], [G LOSS: 172.02]
E: 13, [D ACC: %99.61], [D LOSS: 6.40], [G ACC: %0

E: 111, [D ACC: %100.00], [D LOSS: 0.26], [G ACC: %15.62], [G LOSS: 234.81]
E: 112, [D ACC: %100.00], [D LOSS: 0.21], [G ACC: %6.25], [G LOSS: 196.86]
E: 113, [D ACC: %100.00], [D LOSS: 0.54], [G ACC: %3.12], [G LOSS: 252.26]
E: 114, [D ACC: %100.00], [D LOSS: 0.21], [G ACC: %9.38], [G LOSS: 169.29]
E: 115, [D ACC: %100.00], [D LOSS: 0.45], [G ACC: %31.25], [G LOSS: 133.31]
E: 116, [D ACC: %100.00], [D LOSS: 0.25], [G ACC: %56.25], [G LOSS: 78.98]
E: 117, [D ACC: %100.00], [D LOSS: 0.37], [G ACC: %40.62], [G LOSS: 102.72]
E: 118, [D ACC: %100.00], [D LOSS: 0.15], [G ACC: %62.50], [G LOSS: 72.39]
E: 119, [D ACC: %100.00], [D LOSS: 0.18], [G ACC: %78.12], [G LOSS: 41.02]
E: 120, [D ACC: %100.00], [D LOSS: 0.19], [G ACC: %84.38], [G LOSS: 30.81]
E: 121, [D ACC: %100.00], [D LOSS: 0.12], [G ACC: %93.75], [G LOSS: 28.67]
E: 122, [D ACC: %100.00], [D LOSS: 0.08], [G ACC: %96.88], [G LOSS: 14.88]
E: 123, [D ACC: %100.00], [D LOSS: 0.12], [G ACC: %100.00], [G LOSS: 8.91]
E: 124, [D ACC: %100.0

E: 221, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %100.00], [G LOSS: 5.00]
E: 222, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 3.60]
E: 223, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %100.00], [G LOSS: 5.73]
E: 224, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 5.66]
E: 225, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 3.84]
E: 226, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 4.71]
E: 227, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %100.00], [G LOSS: 5.87]
E: 228, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 4.61]
E: 229, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %100.00], [G LOSS: 6.02]
E: 230, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %100.00], [G LOSS: 4.36]
E: 231, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 7.71]
E: 232, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 3.62]
E: 233, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 3.31]
E: 234, [D ACC: %100.00],

...

E: 770, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 771, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 772, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 773, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 774, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 775, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 776, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 777, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 778, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 779, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 780, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 781, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 782, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 783, [D A

E: 878, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %0.00], [G LOSS: 1426.89]
E: 879, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %12.50], [G LOSS: 732.52]
E: 880, [D ACC: %99.61], [D LOSS: 0.70], [G ACC: %43.75], [G LOSS: 181.33]
E: 881, [D ACC: %100.00], [D LOSS: 0.39], [G ACC: %87.50], [G LOSS: 40.40]
E: 882, [D ACC: %100.00], [D LOSS: 0.10], [G ACC: %100.00], [G LOSS: 0.44]
E: 883, [D ACC: %100.00], [D LOSS: 0.15], [G ACC: %100.00], [G LOSS: 1.52]
E: 884, [D ACC: %100.00], [D LOSS: 0.06], [G ACC: %100.00], [G LOSS: 1.83]
E: 885, [D ACC: %100.00], [D LOSS: 0.11], [G ACC: %100.00], [G LOSS: 0.65]
E: 886, [D ACC: %100.00], [D LOSS: 0.07], [G ACC: %96.88], [G LOSS: 4.39]
E: 887, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %96.88], [G LOSS: 7.36]
E: 888, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %100.00], [G LOSS: 0.43]
E: 889, [D ACC: %100.00], [D LOSS: 0.07], [G ACC: %100.00], [G LOSS: 3.23]
E: 890, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %100.00], [G LOSS: 0.93]
E: 891, [D ACC: %100.00],

E: 988, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %100.00], [G LOSS: 0.47]
E: 989, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 1.00]
E: 990, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 0.56]
E: 991, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 0.86]
E: 992, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %100.00], [G LOSS: 0.64]
E: 993, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 0.59]
E: 994, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 0.45]
E: 995, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.77]
E: 996, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.83]
E: 997, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 0.27]
E: 998, [D ACC: %100.00], [D LOSS: 0.02], [G ACC: %100.00], [G LOSS: 0.75]
E: 999, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.73]
E: 1000, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %100.00], [G LOSS: 0.24]
E: 1001, [D ACC: %100.00

E: 1096, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %100.00], [G LOSS: 0.56]
E: 1097, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.52]
E: 1098, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.30]
E: 1099, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %100.00], [G LOSS: 0.31]
E: 1100, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.28]
E: 1101, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.21]
E: 1102, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %100.00], [G LOSS: 1.09]
E: 1103, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.38]
E: 1104, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.33]
E: 1105, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %100.00], [G LOSS: 0.37]
E: 1106, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.84]
E: 1107, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.57]
E: 1108, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %100.00], [G LOSS: 0.32]
E: 1109, [D 

E: 1204, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 1205, [D ACC: %100.00], [D LOSS: 0.01], [G ACC: %0.00], [G LOSS: 1594.24]
E: 1206, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]
E: 1207, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 1594.24]


In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();